In [ ]:
import os
import pandas as pd
import smtplib
from email.message import EmailMessage

​
​
"""
​
Function to send emails
​
"""
​

In [ ]:

def send_mail_with_excel(recipient_email,
                         subject,
                         content,
                         excel_file,
                         SENDER_EMAIL='ap@radishfiction.com',
                         APP_PASSWORD='radishinvoice'):
    msg = EmailMessage()
    msg['Subject'] = subject
    msg['From'] = SENDER_EMAIL
    msg['To'] = recipient_email
    msg.set_content(content)

    with open(excel_file, 'rb') as f:
        file_data = f.read()
    msg.add_attachment(file_data, maintype="application", subtype="xlsx", filename=excel_file)

    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
        smtp.login(SENDER_EMAIL, APP_PASSWORD)
        smtp.send_message(msg)

In [ ]:
        

def get_excel_file():
    
    # list the name of files in the current directory
    file_list = os.listdir()
    print(file_list)

    # get name of the xlsx file
    xlsx_file_list = [file for file in file_list if '.xlsx' in file]

    if len(xlsx_file_list) < 1: 
        raise ValueError('There is no excel file in the current directory!')
    else:
        xlsx_filename = xlsx_file_list[0]
        print(xlsx_filename)
    
    # read in the excel
    df = pd.read_excel(f'./{xlsx_filename}', header=[0,1,2])
    
    def clean_col_name(s: tuple):
        full_name = ''
        for i, col in enumerate(s):
            if ('Unnamed' in col) | ('Month' in col) | ('Coin Type' in col):
                col = ''
            elif ('free' in col) | ('paid' in col):
                col += ' coin'
            else:
                col = col.replace('Royalty Payment ','').lower()
                # add _ only if it's not the last token
                if i < len(s) - 1:
                    col += '_'

            full_name += col

        return full_name
    
    # reformat column names
    df.columns = [clean_col_name(col) for col in df.columns[:-1]] + ['final dollar amount']
    
    return df

In [ ]:


def main(df):
    
    # make folder for payees' reports
    os.makedirs(f'./payee', exist_ok=True)

    for payee in df.payee.unique():
        # make a folder by the name of payee
        filename = payee.lower()
        os.makedirs(f'./payee/{filename}', exist_ok=True)

        # get report for this payee
        report = df[df.payee == payee]

        # save the report under the payee's folder
        report.to_excel(f'./payee/{filename}/report.xlsx')

        # send the email to payee
        recipient_email = df[df.payee==payee].email.values[0]
        subject = 'Royalty Payment'
        content = """
            Hello, 

            This is the royalty payment from Radish!

            If you have any inquiries, please contact our customer service.

            Best Regards,
        """
        excel_file = f'./payee/{payee}/report.xlsx'

        #send_mail_with_excel(recipient_email, subject, content, excel_file)

        print(f'Finished {payee}...')
        

if __name__ == "__main__":
    main(get_excel_file())